# 1. Собираем нашу первую свёрточную нейросеть 

Пришло время построить нашу первую свёрточную нейросеть. Будем использовать для этого датасет [CIFAR-10](https://paperswithcode.com/sota/image-classification-on-cifar-10). Набор данных включает в себя цветные изображения из 10 различных классов.

<img src="https://paperswithcode.com/media/datasets/CIFAR-10-0000000431-b71f61c0_U5n3Glr.jpg" width="600">

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from tqdm.notebook import tqdm

## 1.1. Смотрим на данные 

Скачаем и приготовим данные. Буквально через минуту в наших руках окажутся $60 000$ ($50 000$ для обучения, $10 000$ для валидации) цветных картинок размера $32 \times 32$.

In [ ]:
from pathlib import Path
from torch.hub import _get_torch_home

# На Linux датасет скачается в ~/.cache/torch/datasets, но можете выбрать любую другую папку
datasets_path = Path(_get_torch_home()) / 'datasets'

dataset_train = torchvision.datasets.CIFAR10(
    datasets_path, train=True, download=True,
    transform=torchvision.transforms.ToTensor()
)
dataset_valid = torchvision.datasets.CIFAR10(
    datasets_path, train=False, download=True,
    transform=torchvision.transforms.ToTensor()
)

print(dataset_train.classes)

Нарисуем несколько рандомных картинок из тренировочной выборки. 

In [ ]:
plt.figure(figsize=(16, 10))
n = 10

random_indices = np.random.choice(range(len(dataset_train)), size=n)

for i, idx in enumerate(random_indices):
    plt.subplot(1, n, i + 1)
    X, y = dataset_train[idx]
    plt.imshow(X.numpy().transpose(1, 2, 0))
    plt.title(dataset_train.classes[y])
    plt.xticks([])
    plt.yticks([])

plt.show()

Заранее создадим даталоадеры:

In [ ]:
batch_size = 500

train_dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size)

## 1.2. Полносвязная сетка 

Соберём полносвязную сетку по аналогии с тем, что мы делали в прошлый раз:

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    <YOUR CODE>
)
# Не забудьте перенести модель на device!

with torch.no_grad():
    y_pred = model(dataset_train[0][0].unsqueeze(0).to(device))
    assert y_pred.shape == (1, len(dataset_train.classes)), 'Модель должна выдавать по логиту для каждого класса'
    del y_pred
    assert next(model.parameters()).device.type == torch.device(device).type, 'Вы забыли перенести модель на device'

model

Кстати, заодно посмотрим на библиотеку `torchsummary`, позволяющую красиво печатать модель:

In [ ]:
from torchsummary import summary

# на вход надо передать шейп входа модели, не считая размерности батча
summary(model, dataset_train[0][0].shape, device=device)

Зафиксируем параметры обучения:

In [ ]:
num_epochs = 20
learning_rate = 1e-3  # Кстати, это learning rate по умолчанию для Adam

Заведём `criterion`, `opt`...

In [ ]:
criterion = <YOUR CODE>

In [ ]:
opt = <YOUR CODE>

Функция для обучения. Ничего необычного:

In [ ]:
def train(model, criterion, opt, train_dataloader, valid_dataloader, num_epochs, device='cuda:0'):
    history = {'loss_train': [], 'loss_valid': [], 'accuracy_valid': [], 'lr': []}
    
    with tqdm(range(1, num_epochs + 1)) as progress_bar:
        for epoch in progress_bar:
            epoch_losses_train = []
            epoch_losses_valid = []
            epoch_correct_predictions_valid = []
            
            # Трейн
            for x_batch, y_batch in train_dataloader:
                # Переносим батч на GPU
                x_batch = <YOUR CODE>
                y_batch = <YOUR CODE>

                y_pred = <YOUR CODE>  # делаем предсказания
                loss = <YOUR CODE>  # считаем лосс
                
                epoch_losses_train.append(loss.item())
                assert np.isfinite(epoch_losses_train[-1])

                # Считаем градиенты и делаем шаг оптимизатора, не забыв обнулить градиенты
                <YOUR CODE>

            with torch.no_grad():
                for x_batch, y_batch in valid_dataloader:
                    # Переносим батч на GPU
                    x_batch = <YOUR CODE>
                    y_batch = <YOUR CODE>

                    y_pred = <YOUR CODE> # делаем предсказания
                    loss = <YOUR CODE> # считаем лосс
                    
                    epoch_losses_valid.append(loss.item())
                    assert np.isfinite(epoch_losses_valid[-1])

                    batch_correct_predictions = torch.argmax(y_pred, dim=-1) == y_batch
                    epoch_correct_predictions_valid.extend(batch_correct_predictions.to('cpu').numpy().tolist())
                    
            history['loss_train'].append(np.mean(epoch_losses_train))
            history['loss_valid'].append(np.mean(epoch_losses_valid))
            history['accuracy_valid'].append(np.mean(epoch_correct_predictions_valid))
            history['lr'].append(opt.param_groups[0]['lr'])

            # выводим статистику
            stats = f'loss: {history["loss_valid"][-1]:.5f}, accuracy: {history["accuracy_valid"][-1]:.4f}'
            print(f'Epoch: {epoch}, {stats}')
            progress_bar.set_postfix_str(stats)
            
    return history

Вспомогательная функция, чтобы рисовать графики:

In [ ]:
def plot_history(histories):
    plt.figure(figsize=(16, 10))

    for name, history in histories.items():
        train = plt.plot(history['loss_train'], label=f'{name} train')
        plt.plot(history['loss_valid'], color=train[0].get_color(), linestyle='--', label=f'{name} valid')

    plt.xlabel('Epochs')
    plt.ylabel('Log loss')
    plt.legend()
    plt.grid()

histories = {}

Учим бейзлайн:

In [ ]:
histories['fc'] = <YOUR CODE>

In [ ]:
plot_history(histories)

## 1.3. Свёрточная сетка 

Свёрточная нейронная сеть строится из нескольких разных типов слоёв: 

* [`nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) — Свёртка:
    - **`in_channels`**: число каналов на входе;
    - **`out_channels`**: число каналов на выходе; 
    - **`kernel_size`**: размер окна для свёртки;
    - **`padding`**: какой ширины будет каёмка из нулей по краям картинки перед непосредственно свёрткой (если хотите, чтобы свёртка не меняла размер картинки, ставьте `padding=(kernel_size - 1) // 2`)
* [`nn.MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) — max pooling
* [`nn.AvgPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html) — average pooling
* [`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.flatten.html) — разворачивает картинку в вектор 
* [`nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) — полносвязный слой (fully-connected layer)
* [`nn.ReLU`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) — функция активации. Естественно, можно выбрать любую другую


В модели, которую мы определим ниже, на вход будут тензоры размера `(B, 1, 32, 32)`, а на выходе `(B, 10)` — это будет вероятность того, что объект относится к конкретному классу. `B`, как обычно, означает размерность батча. 

Теперь давайте соберём свёрточную сеть наподобие LeNet-5: 

* Свёртка с $3$ каналами на входе (для цветного изображения), $32$ каналами на выходе, ядром $5 \times 5$ и `padding` таким, чтобы размер изображения не менялся
* ReLU
* Max-pooling с ядром $2 \times 2$ с шагом (strides) $2$ по обеим осям
* Свёртка с $16$ каналами на выходе, ядром $5 \times 5$ и `padding` таким, чтобы размер изображения не менялся
* ReLU
* Max-pooling с ядром $2 \times 2$ с шагом (strides) $2$ по обеим осям
* `Flatten`
* Три полносвязных слоя с $120$, $60$ и $10$ нейронами соответственно. Здесь вам нужно будет посчитать или посмотреть, какого размера тензоры будут получаться после `Flatten`

Это не то же самое, что оригинальный LeNet-5. Если вы заглянете в [оригинальную статью](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf), то увидите там очень читабельное описание архитектуры, которая на современный взгляд выглядит странно.

Реализацию оригинальной архитектуры на PyTorch можно посмотреть, например, [тут](https://github.com/maorshutman/lenet5-pytorch).

In [ ]:
model = <YOUR CODE>
# Не забудьте перенести модель на device!

with torch.no_grad():
    y_pred = model(dataset_train[0][0].unsqueeze(0).to(device))
    assert y_pred.shape == (1, len(dataset_train.classes)), 'Модель должна выдавать по логиту для каждого класса'
    del y_pred
    assert next(model.parameters()).device.type == torch.device(device).type, 'Вы забыли перенести модель на device'

model

In [ ]:
summary(model, dataset_train[0][0].shape, device=device)

In [ ]:
opt = <YOUR CODE>

In [ ]:
histories['conv'] = <YOUR CODE>

In [ ]:
plot_history(histories)

Как видите, точность довольно сильно подскочила. Попробуйте поиграться числом параметров и слоёв так, чтобы их стало меньше, а качество сетки стало лучше. Попробуйте обучать нейросетку большее количество эпох.

In [ ]:
# Здесь могли быть ваши эксперименты

# 2. Готовые архитектуры

Здесь мы посмотрим на реализации готовых архитектур, о которых мы говорили на лекции, в библиотеке torchvision.

## 2.1. AlexNet

In [ ]:
# можно указать pretrained=True, и тогда torchvision скачает готовые веса, обученные на ImageNet
model = torchvision.models.alexnet()
model

In [ ]:
summary(model, (3, 224, 224), device='cpu')

## 2.2. VGG

In [ ]:
model = torchvision.models.vgg16()
model

In [ ]:
summary(model, (3, 224, 224), device='cpu')

В этом месте давайте вспомним [ноутбук](https://github.com/dniku/neural_nets_dpo/blob/master/week01/pytorch_pretrained_model_demo.ipynb) с самого первого семинара. Там мы как раз использовали VGG-16!

## 2.3. GoogLeNet

In [ ]:
# init_weights=False нужен из-за бага в scipy (попробуйте убрать этот параметр, и увидите предупреждение)
model = torchvision.models.googlenet(init_weights=False)
model

In [ ]:
summary(model, (3, 224, 224), device='cpu')

## 2.4. ResNet

In [ ]:
model = torchvision.models.resnet18(pretrained=False)
model

In [ ]:
summary(model, (3, 224, 224), device='cpu')

# 3. Реализуем ResNet

Здесь мы руками изготовим модель, в точности повторяющую ResNet-18 из `torchvision.models` — настолько, что можно будет взять `state_dict` от одной модели и загрузить в другую. Этот процесс мы разделим на две части.

В первую очередь мы сделаем так называемый residual block: модуль, содержащий внутри себя skip connection. Мы его сделаем так, чтобы при проходе через него у тензора могли измениться размеры или количество каналов. Он выглядит так:

```
--> conv -> bn -> relu -> conv -> bn --> + -->relu -->
 |                                       ↑
 '--------->optionally downsample--------'
```

При этом:

* Все свёртки `conv` имеют kernel size 3x3 и padding=1
* Изменение количества каналов и страйды есть только в первой свёртке `conv`
* `downsample` — это последовательность из свёртки 1x1 (опционально со страйдами) и батчнорма
* Во всех свёртках не используется bias

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1) -> None:
        super(BasicBlock, self).__init__()

        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        <YOUR CODE>

Теперь соберём саму модель. Она состоит из начала вида

```
conv -> bn -> relu -> maxpool
```

Затем 4 раза повторяется конструкция из серии `BasicBlock`. В ResNet-18 в каждой такой серии блоков 2.

```
layer1: BasicBlock(64, 64, stride=1) -> BasicBlock(64, 64, stride=1)
layer2: BasicBlock(64, 128, stride=2) -> BasicBlock(128, 128, stride=1)
layer3: BasicBlock(128, 256, stride=2) -> BasicBlock(256, 256, stride=1)
layer4: BasicBlock(256, 512, stride=2) -> BasicBlock(512, 512, stride=1)
```

Наконец, в конце результат усредняется по пространственным размерностям и применяется один полносвязный слой, чтобы сделать итоговое предсказание. Таким образом, ResNet-18 — это логистическая регрессия поверх свёрточных фичей.

In [ ]:
class ResNet18(nn.Module):
    def __init__(self, num_classes: int =1000) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64)
        self.layer2 = self._make_layer(64, 128, stride=2)
        self.layer3 = self._make_layer(128, 256, stride=2)
        self.layer4 = self._make_layer(256, 512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Это усредняет активации по пространственным размерностям
        self.fc = nn.Linear(512, num_classes)

        # В реализации ResNet-18 из torchvision используется ещё хитрая инициализация весов.
        # Здесь мы это опускаем.

    @staticmethod
    def _make_layer(in_channels: int, out_channels: int, stride: int = 1) -> nn.Sequential:
        return nn.Sequential(
            BasicBlock(in_channels, out_channels, stride),
            BasicBlock(out_channels, out_channels),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        <YOUR CODE>


resnet18 = ResNet18()

Ну и проверим, что она ведёт себя так же, как оригинал.

In [ ]:
resnet18 = ResNet18()
tv_resnet18 = torchvision.models.resnet18()

tv_resnet18.load_state_dict(resnet18.state_dict())

x = torch.randn(1, 3, 224, 224)
torch.allclose(resnet18(x), tv_resnet18(x))

In [ ]:
resnet18 = ResNet18()
tv_resnet18 = torchvision.models.resnet18()

resnet18.load_state_dict(tv_resnet18.state_dict())

x = torch.randn(1, 3, 224, 224)
torch.allclose(resnet18(x), tv_resnet18(x))